In [17]:
# import optuna
# import random
# import numpy as np

# class GASampler(optuna.samplers.BaseSampler):
#     def __init__(self, population_size=10, generations=5, mutation_rate=0.1, crossover_rate=0.8):
#         self.population_size = population_size
#         self.generations = generations
#         self.mutation_rate = mutation_rate
#         self.crossover_rate = crossover_rate

#         self.population = None
#         self.fitness_scores = []
#         self.param_names = []
#         self.param_distributions = {}
#         self.current_individual_index = 0
#         self.current_generation = 0

#     def infer_relative_search_space(self, study, trial):
#         return {}

#     def sample_relative(self, study, trial, search_space):
#         return {}

#     def sample_independent(self, study, trial, param_name, param_distribution):
#         # Record parameter names and distributions
#         if param_name not in self.param_names:
#             self.param_names.append(param_name)
#             self.param_distributions[param_name] = param_distribution

#         # If population not yet initialized, sample random for now
#         if self.population is None:
#             if getattr(param_distribution, "log", False):
#                 value = np.exp(np.random.uniform(np.log(param_distribution.low), np.log(param_distribution.high)))
#             else:
#                 value = np.random.uniform(param_distribution.low, param_distribution.high)
#             return value

#         # Safety check to prevent index out of bounds
#         if self.current_individual_index >= len(self.population):
#             self._evolve_population()
#             self.current_individual_index = 0
#             self.fitness_scores = []

#         # Otherwise, sample from population
#         idx = self.param_names.index(param_name)
#         value = self.population[self.current_individual_index, idx]

#         return value

#     def after_trial(self, study, trial, state, values):
#         """After each trial, record fitness and move to the next individual."""

#         if self.population is None and len(self.param_names) == len(self.param_distributions):
#             self._initialize_population()

#         if values is not None:
#             fitness = values[0]  # Assume single objective optimization
#             self.fitness_scores.append(fitness)

#             self.current_individual_index += 1

#             # If whole population has been evaluated
#             if self.current_individual_index >= self.population_size:
#                 self._evolve_population()
#                 self.current_individual_index = 0
#                 self.fitness_scores = []
#                 self.current_generation += 1

#     def _initialize_population(self):
#         num_params = len(self.param_names)
#         self.population = np.zeros((self.population_size, num_params))

#         for i, name in enumerate(self.param_names):
#             dist = self.param_distributions[name]

#             if getattr(dist, "log", False):
#                 samples = np.exp(np.random.uniform(np.log(dist.low), np.log(dist.high), size=self.population_size))
#             else:
#                 samples = np.random.uniform(dist.low, dist.high, size=self.population_size)

#             self.population[:, i] = samples

#     def _evolve_population(self):
#         if not self.fitness_scores:
#             return  # Skip evolution if no fitness scores available
        
#         # Selection
#         parents = self._select_parents()

#         # Crossover
#         offspring = []
#         for i in range(0, len(parents), 2):
#             if i + 1 < len(parents) and random.random() < self.crossover_rate:
#                 c1, c2 = self._crossover(parents[i], parents[i + 1])
#                 offspring.append(c1)
#                 offspring.append(c2)
#             else:
#                 offspring.append(parents[i])
#                 if i + 1 < len(parents):
#                     offspring.append(parents[i + 1])

#         offspring = np.array(offspring)

#         # Mutation
#         self._mutate(offspring)

#         # Ensure we have exactly population_size individuals
#         if len(offspring) > self.population_size:
#             offspring = offspring[:self.population_size]
#         elif len(offspring) < self.population_size:
#             # If we have fewer offspring than population_size, generate random individuals
#             missing = self.population_size - len(offspring)
#             random_individuals = np.zeros((missing, offspring.shape[1]))
#             for j in range(offspring.shape[1]):
#                 dist = self.param_distributions[self.param_names[j]]
#                 if getattr(dist, "log", False):
#                     samples = np.exp(np.random.uniform(np.log(dist.low), np.log(dist.high), size=missing))
#                 else:
#                     samples = np.random.uniform(dist.low, dist.high, size=missing)
#                 random_individuals[:, j] = samples
#             offspring = np.vstack([offspring, random_individuals])

#         # Replace the old population
#         self.population = offspring

#     def _select_parents(self):
#         # Handle the case where all fitness scores are the same
#         if len(set(self.fitness_scores)) <= 1:
#             indices = np.random.choice(len(self.population), size=self.population_size)
#             return self.population[indices]
        
#         # For minimization problems, invert the scores so smaller values get higher probability
#         fitness_scores = np.array(self.fitness_scores)
#         adjusted_fitness = np.max(fitness_scores) - fitness_scores + 1e-10  # Add small constant to avoid zeros
        
#         total_fitness = np.sum(adjusted_fitness)
#         probs = adjusted_fitness / total_fitness
        
#         # Ensure probabilities sum to 1
#         probs = probs / np.sum(probs)
        
#         indices = np.random.choice(len(self.population), size=self.population_size, p=probs)
#         return self.population[indices]

#     def _crossover(self, parent1, parent2):
#         # Single-point crossover
#         point = np.random.randint(1, len(parent1)) if len(parent1) > 1 else 1
#         child1 = np.concatenate((parent1[:point], parent2[point:]))
#         child2 = np.concatenate((parent2[:point], parent1[point:]))
#         return child1, child2

#     def _mutate(self, offspring):
#         num_params = offspring.shape[1]
#         for i in range(offspring.shape[0]):
#             for j in range(num_params):
#                 if random.random() < self.mutation_rate:
#                     dist = self.param_distributions[self.param_names[j]]
#                     if getattr(dist, "log", False):
#                         offspring[i, j] = np.exp(np.random.uniform(np.log(dist.low), np.log(dist.high)))
#                     else:
#                         offspring[i, j] = np.random.uniform(dist.low, dist.high)

In [2]:
from config import DB_CONN_STRING

In [19]:
# from config import DB_CONN_STRING

# study = optuna.create_study(
#     direction="minimize",
#     study_name="BSA-tuning",
#     storage=DB_CONN_STRING,
#     load_if_exists=True,
#     pruner=optuna.pruners.HyperbandPruner(),
#     sampler=GASampler(
#      population_size=20,
#      generations=5,
#      mutation_rate=0.1,
#      crossover_rate=0.8
#     )
# )

ModuleNotFoundError: No module named 'config'

In [4]:
import optuna


def objective(trial):
    x = trial.suggest_float("x", -1, 1)
    y = trial.suggest_int("y", -1, 1)
    return x**2 + y


sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(
    study_name="CmaEs-testing",
    sampler=sampler,
    storage=DB_CONN_STRING,
    load_if_exists=True,
)
study.optimize(objective, n_trials=20)

[I 2025-05-01 00:36:28,331] A new study created in RDB with name: CmaEs-testing
[I 2025-05-01 00:36:28,435] Trial 0 finished with value: 1.573792195302735 and parameters: {'x': 0.7574907229152941, 'y': 1}. Best is trial 0 with value: 1.573792195302735.
[I 2025-05-01 00:36:28,674] Trial 1 finished with value: -0.9377641075099982 and parameters: {'x': -0.24947122577564296, 'y': -1}. Best is trial 1 with value: -0.9377641075099982.
[I 2025-05-01 00:36:28,799] Trial 2 finished with value: 0.09669616769292676 and parameters: {'x': 0.31096007411390736, 'y': 0}. Best is trial 1 with value: -0.9377641075099982.
[I 2025-05-01 00:36:28,912] Trial 3 finished with value: -0.9939281298651731 and parameters: {'x': 0.07792220565940688, 'y': -1}. Best is trial 3 with value: -0.9939281298651731.
[I 2025-05-01 00:36:29,026] Trial 4 finished with value: 0.007329524900785159 and parameters: {'x': 0.08561264451461104, 'y': 0}. Best is trial 3 with value: -0.9939281298651731.
[I 2025-05-01 00:36:29,136] Tri

In [5]:
import optuna


def objective(trial):
    x = trial.suggest_float("x", -100, 100)
    y = trial.suggest_categorical("y", [-1, 0, 1])
    f1 = x**2 + y
    f2 = -((x - 2) ** 2 + y)
    return f1, f2


# We minimize the first objective and maximize the second objective.
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(directions=["minimize", "maximize"], sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2025-05-01 00:37:53,646] A new study created in memory with name: no-name-87337374-5a6e-4e8b-a08a-d284d6c91f55
[I 2025-05-01 00:37:53,647] Trial 0 finished with values: [51.094200228941936, -83.68628311324875] and parameters: {'x': -7.148020721076705, 'y': 0}.
[I 2025-05-01 00:37:53,648] Trial 1 finished with values: [1500.325997661599, -1349.3898302810112] and parameters: {'x': 38.734041845147004, 'y': 0}.
[I 2025-05-01 00:37:53,648] Trial 2 finished with values: [5276.578631950015, -4989.990851561368] and parameters: {'x': 72.64694509716162, 'y': -1}.
[I 2025-05-01 00:37:53,648] Trial 3 finished with values: [3074.353531262057, -3300.1770117599362] and parameters: {'x': -55.455870124469755, 'y': -1}.
[I 2025-05-01 00:37:53,648] Trial 4 finished with values: [2738.6247095888293, -2533.3354471948373] and parameters: {'x': 52.32231559849802, 'y': 1}.
[I 2025-05-01 00:37:53,649] Trial 5 finished with values: [1663.1581089591489, -1504.0798156661838] and parameters: {'x': 40.7695733232